In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import os, sys, importlib, copy
from glob import glob
import pickle
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import scipy

from utilities import reduced_parameter as red
from field.electromagnetic import FreeVectorPotential, CavityVectorPotential

from matter.atoms import AtomsInBox
from calculator.calculator import Calculator

from utilities.etc import categorizing_result

import animation as an
import analyze_tools.utils as atu
from analyze_tools.monte_carlo import get_colliding_time

from simulation.single import single_collision_simulation
import utilities.reduced_parameter as red
from utilities.etc import moving_average


In [64]:
20 * red.boltzmann / red.epsilon

0.16695708068580803

In [68]:
Lxy = 3e7; Lz = 3e3
T = 20

atoms = AtomsInBox(
    Lxy = Lxy, Lz = Lz, cell_width = (1e6,1e2), 
    mass_dict = red.mass_dict)

v_ar = np.sqrt((T * red.boltzmann / red.epsilon) )
v_xe = - v_ar / np.sqrt(red.mass_dict['Xe'])

atoms.add(
        elements = ["Ar"],
        r = np.array([[0.0, 0, -6/2]]),
        r_dot = np.array([[0.0, 0.0, v_ar]])# / np.sqrt(2)
        )

atoms.add(
        elements = ["Xe"],
        r = np.array([[0.0,0.0,6/2]]),
        r_dot = np.array([[0.00, 0.00, v_xe]])# / np.sqrt(2)
        )

idxAr = atoms.element_idx(element = "Ar")
idxXe = atoms.element_idx(element = "Xe")
epsilon_mat, sigma_mat = red.generate_LJparam_matrix(idxAr = idxAr, idxXe = idxXe)

# calculator to the atoms object
atoms.add_calculator(
    calculator_class = Calculator,
    calculator_kwargs = {
        "epsilon": epsilon_mat, "sigma" : sigma_mat, 
        "positive_atom_idx" : idxXe, "negative_atom_idx" : idxAr,
        "mu0" : red.mu0, "d" : red.d0, "a" : red.a, 'd7':red.d7
    })

atoms.update_distance()

In [69]:
k_vector_int = np.array(
    [[i,0,0] for i in range(1,100)]
    ,dtype=np.float64)

Afield = CavityVectorPotential(
    k_vector_int = k_vector_int, amplitude = 'boltzmann',
    Lxy = Lxy, Lz = Lz, constant_c = red.c, coupling_strength = 20 * Lxy , T = T / red.temp
    )

In [70]:
t, rd = single_collision_simulation(
    0, h = 1e-2, atoms = atoms, field = Afield, potential_threshold = 1e-4,
    max_steps = 5000, verbose = True, record_every = 1)

Cycle: 0, iterations: 1, potential: -2.0596E-04
Cycle: 0, iterations: 2, potential: -2.0727E-04
Cycle: 0, iterations: 3, potential: -2.0859E-04
Cycle: 0, iterations: 4, potential: -2.0993E-04
Cycle: 0, iterations: 5, potential: -2.1127E-04
Cycle: 0, iterations: 6, potential: -2.1262E-04
Cycle: 0, iterations: 7, potential: -2.1398E-04
Cycle: 0, iterations: 8, potential: -2.1535E-04
Cycle: 0, iterations: 9, potential: -2.1673E-04
Cycle: 0, iterations: 10, potential: -2.1813E-04
Cycle: 0, iterations: 11, potential: -2.1953E-04
Cycle: 0, iterations: 12, potential: -2.2094E-04
Cycle: 0, iterations: 13, potential: -2.2237E-04
Cycle: 0, iterations: 14, potential: -2.2380E-04
Cycle: 0, iterations: 15, potential: -2.2525E-04
Cycle: 0, iterations: 16, potential: -2.2670E-04
Cycle: 0, iterations: 17, potential: -2.2817E-04
Cycle: 0, iterations: 18, potential: -2.2965E-04
Cycle: 0, iterations: 19, potential: -2.3114E-04
Cycle: 0, iterations: 20, potential: -2.3264E-04
Cycle: 0, iterations: 21, pot

In [58]:
len(rd['atoms'].observable['t'])

899